# Design Considerations

## Part 1: Identify FIVE design decisions for this system.
Argue for how this design decision is significant to the system, and your plans to analyze this design. If needed, use the dataset provided to you.

### 1. Search Technique
Choosing a search technique is an important design consideration for the retrieval service in the system. The search module is responsible for searching the database of known embeddings to find the most similar samples to the input probe. This search needs to be accurate so that we don't misidentify an intruder as a known employee (i.e. a false positive), but also needs to be efficient so that entrants don't have to wait at the access point for long periods of time for the system to complete its search. Even if we decide that we'll use K-Nearest Neighbors for our search algorithm, there are still two main aspects of the search module to consider: exact vs. approximate search and indexing strategy. Implementing some kind of indexing is important because it allows the data to be organized in a way that makes searching, inserting, deleting, and retrieval more efficient.
- **Exact vs. Approximate:** Exact searches like brute force can be highly accurate and can guarantee that the returned samples are the most similar to the input probe; however, this can be computationally expensive, especially if the embeddings database has many samples or high dimensionality. Approximate searches don't require visiting every single sample in the embeddings database, and therefore are more computationally efficient; however, may be less accurate since they don't guarantee that the optimal solution is identified, since they only search for an adequate solution that's good enough.
- **Indexing Strategies:** Indexing strategies organize the data in a way the improves efficiency for search and retrieval. Some strategies like KD trees can be applied to both exact and approximate searches, while others like local sensitivity hashing (LSH) and hierarchical navigable small world graphs (HNSW) are intended for approximate searches. KD trees partition the data by dimension into "hyperrectangles", allowing for more efficient search since some branches can be eliminated; however, tree depth increases with dimensionality, making this less efficient for high dimensional data. LSH groups similar items into buckets by increasing the probability of hashing collisions for similar items, and is a good alternative for high dimensional data; however, may result in false negatives due to its probablistic nature. HNSW is an accurate, scalable and efficient solution for high dimensional data, where the data is organized into heirarchical layers, with each course layers first and more fine-grained layers towards the bottom; however, storing a graph of all of the layers can be memory intensive.

**Analysis Plan**

To analyze the different search techniques, each variation (brute force, exact KNN,KD trees, LSH, HNSW) can be run on a validation set of known embedding vectors that have been precomputed from the gallery database. The mean average precision can then be calculated for each run, so that the quality of the search outputs (ranked nearest samples) can be compared for each search technique. Additionally, the mean search time per sample will be tracked, so we can determine which technique is the most efficient. The search technique resulting in the best balance of mean average precision and efficiency would be chosen for deployment. Mean average precision is chosen as a comparison metric since it takes into account both the number of positive samples identified, as well as the rank of those samples, unlike precision@k, recall@k, and mean reciprocal rank.

### 2. Similarity Measure
Another important design consideration is the similarity measure used in the Nearest Neighbor search when calculating the distance between two embedding vectors. The distance metric must ensure that higher distances are associated with dissimilar images and lower distances are associated with similar images. Different distance metrics can lead to different results for the same points, so it's important to carefully choose the distance measure. Some popular distance metrics include Manhattan distance (aka "city block" distance), Euclidean distance (aka "straight line" distance), Minkowski distance (a generalized version of Manhattan and Euclidean distances), and cosine similarity which measures the angle between two vectors.

**Analysis Plan**

To analyze the different similarity measures, each variation (Manhattan distance, Euclidean distance, Minkwoski distance with different values for the p parameter, and cosine similarity) can be run on a validation set of known embedding vectors that have been precomputed from the gallery database. The mean average precision can then be calculated for each run, so that the quality of the search outputs (ranked nearest samples) can be compared for the search using each similarity measure. Additionally, the mean search time per sample will be tracked, so we can identify any differences in efficiency between the similarity measures. The similarity measure resulting in the best balance of mean average precision and efficiency would be chosen for deployment.

### 3. Number of Nearest Neighbors (K)
Also used in mAP metric

**Analysis Plan**

TBD

### 4. Embedding Vector Dimensionality
TBD

**Analysis Plan**

TBD

### 5. Potential for Racial Bias
Contrast enhancements/augmentations not the same on all skin tones.

**Analysis Plan**

TBD

### 6. Model Selection
TBD

**Analysis Plan**

TBD

### References
- Pham, A. (2022, April 20). A look into K-Dimensional Trees. Medium. https://medium.com/smucs/a-look-into-k-dimensional-trees-290ec69dffe9
- MyScale. (2023, June 14). HNSW vs KNN: The Ultimate Algorithm Showdown. https://myscale.com/blog/hnsw-vs-knn-algorithm-comparison/
- Ng, A. (2018, April 16). kNN.16 Locality sensitive hashing (LSH) [Video]. YouTube. https://www.youtube.com/watch?v=LqcwaW2YE_c
- Kaushik, S. (2021, January 11). Importance of Distance Metrics in Machine Learning Modelling. Towards Data Science. https://towardsdatascience.com/importance-of-distance-metrics-in-machine-learning-modelling-e51395ffe60d
- Gokte, S. A. (2020, November 17). Most Popular Distance Metrics Used in KNN and When to Use Them. KDnuggets. https://www.kdnuggets.com/2020/11/most-popular-distance-metrics-knn.html

## Part 2: Analyze TWO out of the five design decisions. Add the results of your analysis.

#### Import necessary libraries

In [1]:
# Import all libraries necessary for analysis
import sys
sys.path.insert(1,'../')
from src.search.search import Measure
from src.metrics import RankingMetrics
from pipeline import Pipeline, MULTI_IMAGE_GALLERY_STORAGE
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from collections import Counter
from sklearn.metrics import silhouette_score
from PIL import Image
import matplotlib.cm as cm

#### Precompute all embeddings for all models

In [2]:
# Precompute all embeddings for all 4 models
pipeline_064_018 = Pipeline(image_size=64, architecture='resnet_018',
                        dimension=256, measure=Measure.euclidean, gallery_folder=MULTI_IMAGE_GALLERY_STORAGE)
pipeline_064_034 = Pipeline(image_size=64, architecture='resnet_034',
                        dimension=256, measure=Measure.euclidean, gallery_folder=MULTI_IMAGE_GALLERY_STORAGE)
pipeline_224_018 = Pipeline(image_size=224, architecture='resnet_018',
                        dimension=256, measure=Measure.euclidean, gallery_folder=MULTI_IMAGE_GALLERY_STORAGE)
pipeline_224_034 = Pipeline(image_size=224, architecture='resnet_034',
                        dimension=256, measure=Measure.euclidean, gallery_folder=MULTI_IMAGE_GALLERY_STORAGE)

Calculating embeddings...
Indexing embeddings...
Calculating embeddings...
Indexing embeddings...
Calculating embeddings...
Indexing embeddings...
Calculating embeddings...
Indexing embeddings...


#### Calculate principal components of all embeddings for all 4 models

In [8]:
# Read in embeddings from files for all 4 models, and find top 2 principal components using PCA
model_embeddings = {}
model_pca = {}
for image_size in [64, 224]:
    for arch in ['resnet_018', 'resnet_034']:
        model = f'model_size_{image_size:03}_{arch}'
        pattern = f'../storage/embeddings/{model}/*/*.npy'
        npy_files = glob(pattern)
        embeddings = np.zeros([len(npy_files), 256])
        # Read all files in the model's embeddings folder
        for i in range(len(npy_files)):
            filename = npy_files[i]
            with open(filename, 'rb') as f:
                embeddings[i,:] = np.load(f).T
        # Find principal components
        X_reduced = PCA(n_components=2).fit_transform(embeddings)
        model_pca[model] = X_reduced
        model_embeddings[model] = embeddings

### 1. [Extraction Service] Model Selection

In [3]:
pattern = f'../simclr_resources/probe/*/*.jpg'
probe_files = glob(pattern)
# Get names of the person in each image
true_names = [filename.split('/')[-2] for filename in probe_files]
k = 3
metrics = RankingMetrics(k=k)
pipelines = [pipeline_064_018, pipeline_064_034, pipeline_224_018, pipeline_224_034]
models = ['model_size_064_resnet_018', 'model_size_064_resnet_034', 'model_size_224_resnet_018', 'model_size_224_resnet_034']
relevance_ranks = {m:[] for m in models}
for i in range(len(probe_files)):
    probe = Image.open(probe_files[i])
    for j in range(len(pipelines)):
        pipeline = pipelines[j]
        model = models[j]
        neighbors = pipeline.search_gallery(probe, k)
        predicted_names = [f'{n[1]["firstName"]}_{n[1]["lastName"]}' for n in neighbors]
        relevance_rank = [1 if n == true_names[i] else 0 for n in predicted_names]
        relevance_ranks.get(model,[]).append(relevance_rank)

In [52]:
for m in models:
    print(f'{m}: {metrics.calc_metrics(relevance_ranks.get(m))}')

model_size_064_resnet_018: {'Precision@3': 0.1011011011011011, 'Recall@3': nan, 'MAP@3': 0.056389723056389715, 'MRR@3': 0.20587253920587253}
model_size_064_resnet_034: {'Precision@3': 0.12345679012345678, 'Recall@3': nan, 'MAP@3': 0.08508508508508508, 'MRR@3': 0.21855188521855187}
model_size_224_resnet_018: {'Precision@3': 0.18651985318651987, 'Recall@3': nan, 'MAP@3': 0.11111111111111109, 'MRR@3': 0.36653319986653315}
model_size_224_resnet_034: {'Precision@3': 0.03670337003670337, 'Recall@3': nan, 'MAP@3': 0.01868535201868535, 'MRR@3': 0.07857857857857857}


### 2. [Search Service] Similarity Measure

In [4]:
k = 3
metrics = RankingMetrics(k=k)
def minkowski5(p1, p2): return Measure.minkowski(p1, p2, 5)
def minkowski10(p1, p2): return Measure.minkowski(p1, p2, 10)
measures = [Measure.euclidean, Measure.manhattan, Measure.cosine_similarity, minkowski5, minkowski10]
relevance_ranks = {m:[] for m in measures}
for j in range(len(measures)):
    measure = measures[j]
    pipeline_224_018.set_search_measure(measure)
    ranks = []
    for i in range(len(probe_files)):
        probe = Image.open(probe_files[i])
        neighbors = pipeline_224_018.search_gallery(probe, k)
        predicted_names = [f'{n[1]["firstName"]}_{n[1]["lastName"]}' for n in neighbors]
        relevance_rank = [1 if n == true_names[i] else 0 for n in predicted_names]
        ranks.append(relevance_rank)
    relevance_ranks[measure.__name__] = ranks

In [5]:
for m in measures:
    print(f'{m.__name__}: {metrics.calc_metrics(relevance_ranks.get(m.__name__))}')

euclidean: {'Precision@3': 0.18651985318651987, 'Recall@3': nan, 'MAP@3': 0.11111111111111109, 'MRR@3': 0.36653319986653315}
manhattan: {'Precision@3': 0.18551885218551886, 'Recall@3': nan, 'MAP@3': 0.1111111111111111, 'MRR@3': 0.363697030363697}
cosine_similarity: {'Precision@3': 0.0, 'Recall@3': nan, 'MAP@3': 0.0, 'MRR@3': 0.0}
minkowski5: {'Precision@3': 0.18051384718051383, 'Recall@3': nan, 'MAP@3': 0.11177844511177844, 'MRR@3': 0.34734734734734735}
541


## Additional Analysis
Analysis beyond the minimum two required analyses.

### 3. [Search Service] Number of Nearest Neighbors (K) 

In [10]:
# Analysis 3
# optimal k starts near sqrt(N)
# print(np.sqrt(model_embeddings[model].shape[0])) 

ks = [1, 3, 5, 10, 30, 50]
pipeline_224_018.set_search_measure(Measure.euclidean)
relevance_ranks = {k:[] for k in ks}
for i in range(len(probe_files)):
    probe = Image.open(probe_files[i])
    for k in ks:
        neighbors = pipeline_224_018.search_gallery(probe, k)
        predicted_names = [f'{n[1]["firstName"]}_{n[1]["lastName"]}' for n in neighbors]
        relevance_rank = [1 if n == true_names[i] else 0 for n in predicted_names]
        relevance_ranks.get(k,[]).append(relevance_rank)

In [11]:
for k in ks:
    metrics = RankingMetrics(k=k)
    print(f'k={k}: {metrics.calc_metrics(relevance_ranks.get(k))}')

k=1: {'Precision@1': 0.3163163163163163, 'Recall@1': nan, 'MAP@1': 0.0, 'MRR@1': 0.3163163163163163}
k=3: {'Precision@3': 0.18651985318651987, 'Recall@3': nan, 'MAP@3': 0.11111111111111109, 'MRR@3': 0.36653319986653315}


In [ ]:
# TODO
# - Plots
# - Timing

'''
k=2
{'Precision@2': 0.12362362362362363,
 'Recall@2': nan,
 'MAP@2': 0.05005005005005005,
 'MRR@2': 0.1971971971971972}

k=3
{'Precision@3': 0.1011011011011011,
 'Recall@3': nan,
 'MAP@3': 0.056389723056389715,
 'MRR@3': 0.20587253920587253}

k=5
{'Precision@5': 0.07507507507507508,
 'Recall@5': nan,
 'MAP@5': 0.050250250250250254,
 'MRR@5': 0.21332999666333}

k=10
{'Precision@10': 0.05285285285285285,
'Recall@10': nan,
'MAP@10': 0.04174174174174174,
'MRR@10': 0.2217534995312773}

k=20
{'Precision@20': 0.03398398398398399,
 'Recall@20': nan,
 'MAP@20': 0.028728728728728732,
 'MRR@20': 0.2270195478055815}
 
 k=40
 {'Precision@40': 0.022022022022022022,
 'Recall@40': nan,
 'MAP@40': 0.0195945945945946,
 'MRR@40': 0.22956773017229987}

 k=50
 {'Precision@50': 0.01861861861861862,
 'Recall@50': nan,
 'MAP@50': 0.01667667667667668,
 'MRR@50': 0.2301259492510146}
 '''
def minkowski5(p1, p2): return Measure.minkowski(p1, p2, 5)
print(minkowski5.__name__)